In [1]:
import pandas as pd
import numpy as np
import os
from src.data_loader import read_txt_files
from tqdm import tqdm
import pickle

In [2]:
hindi_wiki_path = '/Users/aditya/Documents/self_learning/ERA V3/week 11/train'

data_list = read_txt_files(folder_path=hindi_wiki_path, clean_text=True, data_percentage_limit=0.01)

  1%|          | 1378/137823 [00:00<01:36, 1418.44it/s]


In [3]:
data_list[:5]

["अग्रहार बेलगुलि कर्नाटक के हासन जिले का एक गाँव है । ' अग्रहार ' शब्द इस बात का परिचायक है कि इस गाँव में बड़ी संख्या में ब्राह्मण पण्दित निवास करते थे जो षड्कर्मों को पूरा करते थे । किन्तु वर्तमान समय में वहाँ केवल तीनचार ब्राह्मण परिवार ही हैं ।",
 'निर्देशांक : 2730′ 7924′ 27.5 79.4 27.5; 79.4 निनौरा संकलपुर फर्रुखाबाद , फर्रुखाबाद , उत्तर प्रदेश स्थित एक गाँव है ।',
 'गैरी स्नाइडर अमेरिकी बीट जनरेशन तथा सनफ्रान्सिसको रेनेसाँस के कवि , आलोचक और परिवेशवादी हैं । कविताके लिये उनको पुलित्जर पुरस्कारसे नवाजा गया है । वे प्राचीन चीनि भाषा तथा आधुनिक जापानि कविताके आनुवादक भी हैं । उनका मननचिन्तन जेन बौद्ध धर्मसे प्रभावित है । ऐलन गिंसबर्गसे पहले वे भारत आ चुके हैं । भारत भ्रमण पर उनके पुस्तक का नाम है पैसेज थ्रु इनडिया । उनके उस समय के पत्नी जोयाने कयगर ने भी एक स्मृतिलेख लिखे हैं जिसका नाम है स्ट्रेंज बिग मुन : जापान ऐन्ड इनडिया जर्नलस ।',
 'दौसा जिला भारत के राजस्थान प्रान्त का एक जिला है । जिले का मुख्यालय दौसा में है । यह राजस्थान के जयपुर संभाग में है । 3,432 वर्ग किमी क्षेत्र में

In [4]:
complete_text = ' '.join(data_list)

## Custom Byte Encoder

In [5]:
unique_chars = list(set(complete_text))
len(unique_chars)

115

In [6]:
vocab = sorted(unique_chars)
initial_vocab_size = len(vocab)
stoi = {char: i for i, char in enumerate(vocab)}
itos = {i: char for i, char in enumerate(vocab)}

In [7]:
print(stoi)

{' ': 0, '!': 1, '"': 2, "'": 3, '(': 4, ')': 5, ',': 6, '.': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, ':': 18, ';': 19, '?': 20, 'ँ': 21, 'ं': 22, 'ः': 23, 'अ': 24, 'आ': 25, 'इ': 26, 'ई': 27, 'उ': 28, 'ऊ': 29, 'ऋ': 30, 'ऍ': 31, 'ऎ': 32, 'ए': 33, 'ऐ': 34, 'ऑ': 35, 'ऒ': 36, 'ओ': 37, 'औ': 38, 'क': 39, 'ख': 40, 'ग': 41, 'घ': 42, 'ङ': 43, 'च': 44, 'छ': 45, 'ज': 46, 'झ': 47, 'ञ': 48, 'ट': 49, 'ठ': 50, 'ड': 51, 'ढ': 52, 'ण': 53, 'त': 54, 'थ': 55, 'द': 56, 'ध': 57, 'न': 58, 'ऩ': 59, 'प': 60, 'फ': 61, 'ब': 62, 'भ': 63, 'म': 64, 'य': 65, 'र': 66, 'ऱ': 67, 'ल': 68, 'ळ': 69, 'व': 70, 'श': 71, 'ष': 72, 'स': 73, 'ह': 74, '़': 75, 'ऽ': 76, 'ा': 77, 'ि': 78, 'ी': 79, 'ु': 80, 'ू': 81, 'ृ': 82, 'ॄ': 83, 'ॅ': 84, 'ॆ': 85, 'े': 86, 'ै': 87, 'ॉ': 88, 'ॊ': 89, 'ो': 90, 'ौ': 91, '्': 92, 'ॐ': 93, '॔': 94, 'ॠ': 95, '।': 96, '॥': 97, '॰': 98, 'ॲ': 99, '—': 100, '‘': 101, '’': 102, '“': 103, '”': 104, '†': 105, '•': 106, '…': 107, '′': 108, '″': 109, '⁄': 110,

In [8]:
tokens = [stoi[c] for c in complete_text] # convert to a list of integers using custom encoder
initial_token_len = len(tokens)

In [9]:
print(f'Initially: Number of tokens: {initial_token_len}')
print(f'Initially: Vocab size: {initial_vocab_size}')

# Initially Number of tokens: 11169983
# Initially Vocab size: 125

Initially: Number of tokens: 2349074
Initially: Vocab size: 115


## BPE

In [10]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [11]:
def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [12]:
vocab_size = 5000 # the desired final vocabulary size
num_merges = vocab_size - initial_vocab_size
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in tqdm(range(num_merges)):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  try:
    pair_char = "".join([itos[pair[0]], itos[pair[1]]])
  except KeyError:
    print(f"pair {pair} not in vocab")
    break
  stoi[pair_char] = initial_vocab_size + i
  itos[initial_vocab_size + i] = pair_char
  idx = initial_vocab_size + i
  # print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

# 100%|██████████| 1385/1385 [10:12<00:00,  2.26it/s]
# tokens length: 2349074
# ids length: 847385
# compression ratio: 2.77X

100%|██████████| 4885/4885 [29:28<00:00,  2.76it/s]

tokens length: 2349074
ids length: 630144
compression ratio: 3.73X


In [13]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = [stoi[c] for c in text]
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

In [14]:
print(encode("मैंने किताब लिया और पढ़ाई किया।"))

[505, 22, 175, 263, 4192, 643, 144, 2724, 77, 27, 3209, 96]


In [15]:
def decode(ids):
  text = "".join([itos[idx] for idx in ids])
  return text

In [16]:
decode([505, 22, 175, 263, 4192, 643, 144, 2724, 77, 27, 3209, 96])

'मैंने किताब लिया और पढ़ाई किया।'

In [144]:
for idx in [505, 22, 175, 263, 194, 286, 643, 144, 2724, 77, 27, 115, 271, 96]:
    print(idx,">>",itos[idx])

505 >> मै
22 >> ं
175 >> ने
263 >>  कि
194 >> ता
286 >> ब 
643 >> लिया 
144 >> और 
2724 >> पढ़
77 >> ा
27 >> ई
115 >>  क
271 >> िया
96 >> ।


## Saving artifacts

In [17]:
import json

#Save stoi as json file
file_name = "artifacts/itos.json"
with open(file_name, "w", encoding="utf-8") as json_file:
    json.dump(itos, json_file, ensure_ascii=False, indent=4)

#Save itos as json file
file_name = "artifacts/stoi.json"
with open(file_name, "w", encoding="utf-8") as json_file:
    json.dump(stoi, json_file, ensure_ascii=False, indent=4)    

#Save merges as json file
file_name = "artifacts/merges.pkl"
with open(file_name, 'wb') as f:
    pickle.dump(merges, f)